In [3]:
import requests
import pandas as pd
import spacy

In [4]:
nlp = spacy.load("en_core_web_sm")
nlp.disable_pipes('ner', 'parser')

['ner', 'parser']

In [5]:
# text is Walter Pater's "The Renaissance: Studies in Art and Poetry"
response = requests.get('https://www.gutenberg.org/cache/epub/2398/pg2398.txt') 
text = response.text

In [6]:
text[:300]

'\ufeffThe Project Gutenberg eBook of The Renaissance: Studies in Art and Poetry\r\n    \r\nThis ebook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\no'

In [7]:
text.find('Many attempts have been made by writers on art and poetry to define')

1127

In [8]:
text.find('*** END OF THE PROJECT GUTENBERG EBOOK THE RENAISSANCE: STUDIES IN ART AND POETRY ***')

349683

In [9]:
start = 1127
end = 349683 -1

In [10]:
renaissance = text[start:end]

In [11]:
renaissance_paras = renaissance.split('\r\n\r\n')

In [12]:
author = []
title = []

In [13]:
for para in renaissance_paras:
    author.append('Walter Pater')
    title.append('The Renaissance: Studies in Art and Poetry')

In [14]:
renaissance_df = pd.DataFrame(list(zip(author, title, renaissance_paras)), columns=['author', 'title', 'text'])

In [15]:
renaissance_df.head()

,author,title,text
0,Walter Pater,The Renaissance: Studies in Art and Poetry,Many attempts have been made by writers on art...
1,Walter Pater,The Renaissance: Studies in Art and Poetry,"""To see the object as in itself it really is,""..."
2,Walter Pater,The Renaissance: Studies in Art and Poetry,"The aesthetic critic, then, regards all the ob..."
3,Walter Pater,The Renaissance: Studies in Art and Poetry,"What is important, then, is not that the criti..."
4,Walter Pater,The Renaissance: Studies in Art and Poetry,Often it will require great nicety to disengag...


In [16]:
def process_text(text):
    """Remove new line characters and lemmatize text. Returns string of lemmas"""
    text = text.replace('\n', ' ')
    doc = nlp(text)
    tokens = [token for token in doc]
    no_stops = [token for token in tokens if not token.is_stop]
    no_punct = [token for token in no_stops if token.is_alpha]
    lemmas = [token.lemma_ for token in no_punct]
    lemmas_lower = [lemma.lower() for lemma in lemmas]
    lemmas_string = ' '.join(lemmas_lower)
    return lemmas_string

In [17]:
renaissance_df['lemmas'] = renaissance_df['text'].apply(process_text)

In [18]:
renaissance_df.head()

,author,title,text,lemmas
0,Walter Pater,The Renaissance: Studies in Art and Poetry,Many attempts have been made by writers on art...,attempt writer art poetry define beauty abstra...
1,Walter Pater,The Renaissance: Studies in Art and Poetry,"""To see the object as in itself it really is,""...",object justly say aim true criticism aesthetic...
2,Walter Pater,The Renaissance: Studies in Art and Poetry,"The aesthetic critic, then, regards all the ob...",aesthetic critic regard object work art fair f...
3,Walter Pater,The Renaissance: Studies in Art and Poetry,"What is important, then, is not that the criti...",important critic possess correct abstract defi...
4,Walter Pater,The Renaissance: Studies in Art and Poetry,Often it will require great nicety to disengag...,require great nicety disengage virtue commoner...


In [19]:
renaissance_df.to_csv('pater_dataframe.csv', index=False)

In [20]:
from collections import defaultdict
import wget
from gensim import corpora, models
import pandas as pd
import pyLDAvis.gensim
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [26]:
df = pd.read_csv('pater_dataframe_clean.csv')

In [27]:
df.head()

,Unnamed: 0,author,title,text,lemmas
0,0,Walter Pater,The Renaissance: Studies in Art and Poetry,Many attempts have been made by writers on art...,attempt writer art poetry define beauty abstra...
1,1,Walter Pater,The Renaissance: Studies in Art and Poetry,"""To see the object as in itself it really is,""...",object justly say aim true criticism aesthetic...
2,2,Walter Pater,The Renaissance: Studies in Art and Poetry,"The aesthetic critic, then, regards all the ob...",aesthetic critic regard object work art fair f...
3,3,Walter Pater,The Renaissance: Studies in Art and Poetry,"What is important, then, is not that the criti...",important critic possess correct abstract defi...
4,4,Walter Pater,The Renaissance: Studies in Art and Poetry,Often it will require great nicety to disengag...,require great nicety disengage virtue commoner...


In [28]:
documents = df['lemmas'].to_list()

In [29]:
type(documents)

list

In [30]:
texts =[
    [word for word in document.lower().split()]
    for document in documents
]

In [31]:
frequency = defaultdict(int)
for text in texts:
  for token in text:
    frequency[token] += 1

In [32]:
texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

In [33]:
dictionary = corpora.Dictionary(texts)

In [34]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [35]:
lda_model = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20, passes=50)

In [36]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10    -0.059419 -0.064706       1        1  14.801749
12    -0.024188 -0.071282       2        1  10.393791
13     0.006604 -0.088810       3        1   8.355389
1      0.028413 -0.064916       4        1   8.233945
5      0.027502 -0.083637       5        1   7.591445
9      0.055927 -0.044605       6        1   6.870147
18     0.075984 -0.076451       7        1   6.723910
11    -0.128373  0.042853       8        1   4.914348
4     -0.157474 -0.027900       9        1   4.203742
17    -0.015213 -0.056486      10        1   4.183060
7     -0.045653  0.016207      11        1   3.599365
15    -0.018576 -0.015850      12        1   3.486781
16     0.009546  0.015838      13        1   3.043284
2      0.092289  0.042705      14        1   2.631017
6     -0.070733  0.023313      15        1   2.490416
8      0.165241  0.070745      16        1   2.282560
0      0.072003 -0.000046      17        1   2.273981
14    -0.039612  0.151906      18        1   1.713871
3     -0.020153  0.159885      19        1   1.158155
19     0.045888  0.071238      20        1   1.049042, topic_info=            Term        Freq       Total Category  logprob  loglift
291  renaissance   46.000000   46.000000  Default  30.0000  30.0000
674        amile   32.000000   32.000000  Default  29.0000  29.0000
159         work  176.000000  176.000000  Default  28.0000  28.0000
119           et   31.000000   31.000000  Default  27.0000  27.0000
444          old   67.000000   67.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
744       moment    1.329412   52.349553  Topic20  -5.2032   0.8841
442          new    1.329412   54.597197  Topic20  -5.2032   0.8420
229      strange    1.329413   56.771709  Topic20  -5.2032   0.8030
246      century    1.278860   58.121545  Topic20  -5.2420   0.7407
37         thing    1.329412   86.032293  Topic20  -5.2032   0.3873

[1382 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
463       1  0.476137   abelard
463       6  0.238068   abelard
463      13  0.238068   abelard
463      18  0.079356   abelard
772       9  0.394835     abide
...     ...       ...       ...
310      19  0.031266     youth
1886      7  0.203524  youthful
1886      9  0.203524  youthful
1886     16  0.203524  youthful
1886     18  0.407049  youthful

[4186 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 13, 14, 2, 6, 10, 19, 12, 5, 18, 8, 16, 17, 3, 7, 9, 1, 15, 4, 20])